# XGBoost
## Alex Chen

$var_1$ = Open

$var_2$ = Close

# Import Library

In [142]:
#library pengolahan data
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta

#library sklearn untuk modelling.
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

#library error
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# library xgboost
from xgboost import XGBRegressor


# Tarik data

In [143]:
yf.pdr_override()
df = pdr.get_data_yahoo(["BBCA.JK"])

[*********************100%%**********************]  1 of 1 completed


In [144]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-04-01,10075.0,10100.0,9825.0,9850.0,9850.0,79248700
2024-04-02,9925.0,9950.0,9800.0,9900.0,9900.0,76763100
2024-04-03,9750.0,9800.0,9525.0,9525.0,9525.0,177418400
2024-04-04,9575.0,9850.0,9575.0,9850.0,9850.0,75993500
2024-04-05,9950.0,9950.0,9825.0,9825.0,9825.0,70428600


In [145]:
df.shape

(4912, 6)

# Edit Data sesuai research

In [146]:
df_xg = df.iloc[:,:4]
days=3
df1 = pd.DataFrame()
for j in range(days+1):
    for i in df_xg.columns:
        if j==0:
            df1[f'{i}'] = df_xg[i].iloc[j:]
        else:
            df1[f'{i}_t-{j}'] = df_xg[i].shift(j)

In [148]:
df1.loc[:,['Open']+list(df1.columns[4:])].tail(5)

,Open,Open_t-1,High_t-1,Low_t-1,Close_t-1,Open_t-2,High_t-2,Low_t-2,Close_t-2,Open_t-3,High_t-3,Low_t-3,Close_t-3
Date,,,,,,,,,,,,,
2024-04-01,10075.0,10025.0,10100.0,9975.0,10075.0,10025.0,10075.0,10000.0,10075.0,10050.0,10050.0,9950.0,10050.0
2024-04-02,9925.0,10075.0,10100.0,9825.0,9850.0,10025.0,10100.0,9975.0,10075.0,10025.0,10075.0,10000.0,10075.0
2024-04-03,9750.0,9925.0,9950.0,9800.0,9900.0,10075.0,10100.0,9825.0,9850.0,10025.0,10100.0,9975.0,10075.0
2024-04-04,9575.0,9750.0,9800.0,9525.0,9525.0,9925.0,9950.0,9800.0,9900.0,10075.0,10100.0,9825.0,9850.0
2024-04-05,9950.0,9575.0,9850.0,9575.0,9850.0,9750.0,9800.0,9525.0,9525.0,9925.0,9950.0,9800.0,9900.0


# Train Test Split Data

In [149]:
X = df1.iloc[-400:,]
X = X.loc[:,list(df1.columns[4:])]
y = df1.iloc[-400:,3:4]

In [150]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,shuffle=True,random_state=33)

In [151]:
x_train.columns

Index(['Open_t-1', 'High_t-1', 'Low_t-1', 'Close_t-1', 'Open_t-2', 'High_t-2',
       'Low_t-2', 'Close_t-2', 'Open_t-3', 'High_t-3', 'Low_t-3', 'Close_t-3'],
      dtype='object')

# Bangkitkan Model

Percobaan 1 Tuning Parameter : n_estimators=140, learning_rate = 0.07,objective='reg:squarederror',max_depth=7,min_child_weight=1,subsample=0.9,colsample_bytree=0.9

Percobaan 2 Tuning Parameter : n_estimators=160, learning_rate = 0.07,objective='reg:squarederror',max_depth=8,min_child_weight=1,subsample=0.9,colsample_bytree=0.9

Percobaan 3 Tuning Parameter : n_estimators=200, learning_rate = 0.07,objective='reg:squarederror',max_depth=8,min_child_weight=1,subsample=0.9,colsample_bytree=0.9

Percobaan 4 Tuning Parameter : n_estimators=250, learning_rate = 0.07,objective='reg:squarederror',max_depth=10,min_child_weight=1,subsample=0.9,colsample_bytree=0.9

Percobaan 5 Tuning Parameter : n_estimators=320, learning_rate = 0.07,objective='reg:squarederror',max_depth=20,min_child_weight=1,subsample=0.9,colsample_bytree=0.9

Percobaan 6 Tuning Parameter : n_estimators=320, learning_rate = 0.07,objective='reg:squarederror',max_depth=14,subsample=0.9,colsample_bytree=0.9

Percoban 7 Tuning Parameter : n_estimators=270, learning_rate = 0.07,objective='reg:squarederror',max_depth=14,subsample=0.9,colsample_bytree=0.9

In [152]:
mod2 = XGBRegressor(n_estimators=160, learning_rate = 0.07,objective='reg:squarederror',max_depth=8,min_child_weight=1,subsample=0.9,colsample_bytree=0.9)

mod2.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.07, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=160, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

# Evaluasi Model

In [153]:
print(mod2.score(x_train,y_train))
print(mod2.score(x_test,y_test))

#mod2.predict(x_test)

0.9999063068566146
0.9596137569942227


In [154]:
prediction = mod2.predict(X.tail(30).values)
print(prediction)

[ 9973.916   9875.951   9928.706   9798.401   9872.203   9851.206
  9879.148   9948.381   9751.225   9802.876   9948.365  10114.954
 10149.054  10066.4375 10041.612  10146.725  10147.479  10066.444
 10142.114  10124.787  10143.724  10074.57   10050.514   9902.84
 10074.632   9856.546   9918.075   9526.88    9848.427   9683.991 ]


In [155]:
print(mean_absolute_error(y.tail(30),prediction))
mean_absolute_percentage_error(y.tail(30),prediction)

42.36569010416667


0.004217376201702532

mae = 42

In [166]:
n=300
prediction = mod2.predict(X.tail(n).values)
open_10 = df.Open.tail(n)
index_df = y.tail(n).index
real_10 = y.tail(n).values.flatten()
pred_10 = prediction.flatten()

In [167]:
result_df = pd.DataFrame({'y_real':real_10,'y_pred':pred_10,'open_price':open_10},index=index_df)
result_df['sign_real'] = np.where(result_df.y_real - result_df.y_real.shift(1)>0,1,0)
result_df['sign_prediction'] = np.where(result_df.y_pred - result_df.y_pred.shift(1)>0,1,0)
result_df['same_sign'] = np.where(result_df['sign_prediction']==result_df['sign_real'],1,0)
result_df['error'] = result_df['y_real'] - result_df['y_pred']

result_df['action'] = np.where(result_df.y_pred>result_df.open_price+100,1,0)
#result_df.same_sign.sum()/len(result_df)

In [168]:
result_df

,y_real,y_pred,open_price,sign_real,sign_prediction,same_sign,error,action
Date,,,,,,,,
2023-01-04,8350.0,8364.925781,8525.0,0,0,1,-14.925781,0
2023-01-05,8250.0,8397.789062,8350.0,0,1,0,-147.789062,0
2023-01-06,8300.0,8299.176758,8100.0,1,0,0,0.823242,1
2023-01-09,8450.0,8447.515625,8325.0,1,1,1,2.484375,1
2023-01-10,8175.0,8180.660156,8400.0,0,0,1,-5.660156,0
...,...,...,...,...,...,...,...,...
2024-04-01,9850.0,9856.545898,10075.0,0,0,1,-6.545898,0
2024-04-02,9900.0,9918.075195,9925.0,1,1,1,-18.075195,0
2024-04-03,9525.0,9526.879883,9750.0,0,0,1,-1.879883,0


In [169]:
buy_df = result_df.loc[result_df.action==1]
buy_df['price_chg'] = buy_df['y_real'] - buy_df['open_price']
buy_df['net_profit'] = -(buy_df['open_price']+25)*(1+0.19/100)*100 + (buy_df['y_real']-25) * (1-0.29/100)*100

buy_df

/var/folders/_8/r2326rms4l37yskjs2yv3mv80000gn/T/ipykernel_2923/1628889218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_df['price_chg'] = buy_df['y_real'] - buy_df['open_price']
/var/folders/_8/r2326rms4l37yskjs2yv3mv80000gn/T/ipykernel_2923/1628889218.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_df['net_profit'] = -(buy_df['open_price']+25)*(1+0.19/100)*100 + (buy_df['y_real']-25) * (1-0.29/100)*100


,y_real,y_pred,open_price,sign_real,sign_prediction,same_sign,error,action,price_chg,net_profit
Date,,,,,,,,,,
2023-01-06,8300.0,8299.176758,8100.0,1,0,0,0.823242,1,200.0,11056.50
2023-01-09,8450.0,8447.515625,8325.0,1,1,1,2.484375,1,125.0,3470.25
2023-01-16,8150.0,8151.612793,8050.0,1,1,1,-1.612793,1,100.0,1109.50
2023-01-17,8325.0,8321.916992,8175.0,1,1,1,3.083008,1,150.0,6035.00
2023-01-26,8475.0,8466.392578,8225.0,1,1,1,8.607422,1,250.0,15982.00
2023-02-03,8700.0,8697.263672,8550.0,1,1,1,2.736328,1,150.0,5855.00
2023-02-23,8725.0,8725.022461,8625.0,1,1,1,-0.022461,1,100.0,833.50
2023-02-27,8775.0,8847.256836,8675.0,1,1,1,-72.256836,1,100.0,809.50
2023-03-24,8825.0,8813.335938,8700.0,1,1,1,11.664062,1,125.0,3290.25


net profit with open = 281347.64999999997
net profit w/o open = 281347.64999999997

In [170]:
buy_df.net_profit.sum()

123893.49999999895